# Intelligent Mammogram Mass Analysis and Segmentation (IMMAS)

## Module import

In [ ]:
%matplotlib inline

import immas
import matplotlib.pyplot as plt

In [ ]:
immas.test()

## Reading files

This example shows how to read one image and create MammogramImage file

In [ ]:
from immas import MammogramImage

path_image = "../dataset/images/20587080_b6a4f750c6df4f90_MG_R_ML_ANON.tif"
path_mask = "../dataset/masks/20587080_b6a4f750c6df4f90_MG_R_ML_ANON.png"
pectoral_muscle = "../dataset/pectoral_muscle_masks/20587080_b6a4f750c6df4f90_MG_R_ML_ANON.tif"

# contructor will automatically read data, if contrary not specified
# pectoral muscle will be removed from image if we have corresponding mask available
mm = MammogramImage(path_image, path_mask, pmuscle_mask_path=pectoral_muscle)

# check whether this image has ground truth segmentation result available
if mm.has_masses:
    print('Masses are present on ground truth image')
else:
    print('No masses on ground truth image are present')

# attribute .image_data contains actual image cropped according to the masks    
plt.imshow(mm.image_data, cmap="gray")
plt.axis('off')
plt.title('Cropped image')
plt.show()

#attribute .image_data contains actual image cropped according to the masks   
plt.imshow(mm.uncropped_image, cmap="gray")
plt.axis('off')
plt.title('Uncropped image')
plt.show()

plt.imshow(mm.image_ground_truth, cmap="gray")
plt.axis('off')
plt.title('Ground truth image')
plt.show()

This example shows how to read whole dataset

In [ ]:
from immas.io import read_dataset
import cv2

data_set = read_dataset(image_folder="../dataset/images",
            mask_folder="../dataset/masks",
            results_folder="../dataset/groundtruth",
            pmuscle_mask_folder="../dataset/pectoral_muscle_masks")

print("Number of images for training is {0}, number of images for testing is {1}".format(
    len(data_set["train"]), len(data_set["test"])))

In [ ]:
# let's show an example image from train subset of images, that has also a ground truth image
for img in data_set["train"]:
    mammogram = img
    mammogram.read_data()
    break


plt.imshow(mammogram.image_data, cmap="gray")
plt.axis('off')
plt.title('Cropped image')
plt.show()

plt.imshow(mammogram.uncropped_image, cmap="gray")
plt.axis('off')
plt.title('Uncropped image')
plt.show()

plt.imshow(mammogram.image_ground_truth, cmap="gray")
plt.axis('off')
plt.title('Ground truth')
plt.show()

## Enhancement - PreProcessing

In [ ]:
# Defines function to apply preprocessing techniques.
# This can be used to try different parameters for the preprocessing functions.
# Also can be used to try those functions in different orders.

from immas import preprocessing

def testPreProcessing (img):
    img = preprocessing.resize(img)
    #img = preprocessing.open(img)
    #img = preprocessing.close(img)
    #img = preprocessing.erode(img)
    #img = preprocessing.dilate(img)
    #img = preprocessing.clahe(img)
    img = preprocessing.morphoEnhancement(img,2)
    #img = preprocessing.waveletTransform(img)#.astype('uint16')
    return img

In [ ]:
# Applies the preprocessing techniques defined above in the entire dataset (both training and test sets)

img = data_set["train"]
for m in img:
    m.read_data()
    m.image_data = testPreProcessing(m.image_data)
    #plt.imshow(m.image_data, cmap="gray")
    plt.show()

img = data_set["test"]
for m in img:
    m.read_data()
    m.image_data = preprocessing.fullPreprocessing(m.image_data)

# plt.imshow(m.image_data, cmap="gray")
# cv2.imwrite('preprocessed_img.png',m.image_data)

### Applies preprocessing to a small subset and saves the results as png files

In [ ]:
from immas.io import read_dataset
from immas import preprocessing
import numpy as np
import cv2
import os

data_set = read_dataset(image_folder="../dataset/masses_examples",
            mask_folder="../dataset/masks",
            results_folder="../dataset/groundtruth",
            pmuscle_mask_folder="../dataset/pectoral_muscle_masks")

print("Number of images for training is {0}, number of images for testing is {1}".format(
    len(data_set["train"]), len(data_set["test"])))

savepath = "../dataset/masses_examples_preprocessed"

img = data_set["train"]
for m in img:
    m.read_data()
    m.image_data = testPreProcessing(m.image_data)
    cv2.imwrite(os.path.join(savepath, m.file_name) + '.tif', m.image_data)
    
img = data_set["test"]
for m in img:
    m.read_data()
    m.image_data = testPreProcessing(m.image_data)
    cv2.imwrite(os.path.join(savepath, m.file_name) + '.tif', m.image_data)